# Linpackの実行

このnotebookでは、[070-Linpackフロントエンドの作成](./070-Linpackフロントエンドの作成.ipynb)で作成したLinpackフロントエンドからジョブを実行する方法について示します。また、mdx VM上での実行結果についても示します。

## 前提

このnotebookでは、[070-Linpackフロントエンドの作成](./070-Linpackフロントエンドの作成.ipynb)を実行済みであることを前提とします。

## Linpackジョブ実行方法

Open OnDemandダッシュボードのメニューバーの、"Interactive Apps"をクリックすると、"High-Performance Linpack"のメニューが現われるので、それを選択します。

![](./images/hpl-dashboard.png)

すると、ジョブのパラメータを設定するフォームが表示されます。

![](./images/hpl-params.png)

HPL.datの各パラメータの詳細については、[HPL Tuning](https://www.netlib.org/benchmark/hpl/tuning.html)を参照してください。

パラメータを設定したら、Launchボタンをクリックします。これにより、ジョブがsubmitされます。

Launchボタンをクリックすると、セッションの表示画面に遷移します。"High-Performance Linpack"の右側に、ジョブの状態が表示されます。ジョブの状態は以下のように遷移します。
* Queued
* Running
* Completed

![](./images/hpl-status.png)

実行が完了すると、ジョブの状態がCompletedと表示されます。

ジョブの実行結果を確認するには、Session IDをクリックします。すると、このジョブで使用したディレクトリの内容の画面が表示されます。

![](./images/hpl-result.png)

この画面の、`output.log`がジョブ出力結果のファイルなので、それをクリックすれば、出力結果の内容を確認することができます。

## mdx VM上でのベンチマーク結果

本章では、mdx VMを用いて、ここまでの手順で作成されたSlurmクラスタ上でLinpack (HPL)を実行した結果について示します。

Slurmクラスタの構成については以下の通りです。
* 計算ノードのパック数(CPU数): 3
* 計算ノードの物理メモリ量: 4463MiB
* GPUなし
* CPU(VMの/proc/cpuinfoより): Intel(R) Xeon(R) Platinum 8368 CPU @ 2.40GHz
* 計算ノード数: 4
* ノード間MPI通信: TCP/IP

HPLのパラメータ設定(`HPL.dat`に設定する値)は以下の通りです。

```
384          NBs
0            PMAP process mapping (0=Row-,1=Column-major)
16.0         threshold
2            PFACTs (0=left, 1=Crout, 2=Right)
2            NBMINs (>= 1)
2            NDIVs
1            # of recursive panel fact.
1            RFACTs (0=left, 1=Crout, 2=Right)
0            BCASTs (0=1rg,1=1rM,2=2rg,3=2rM,4=Lng,5=LnM)
0            DEPTHs (>=0)
0            SWAP (0=bin-exch,1=long,2=mix)
1            swapping threshold
1            L1 in (0=transposed,1=no-transposed) form
1            U  in (0=transposed,1=no-transposed) form
0            Equilibration (0=no,1=yes)
8            memory alignment in double (> 0)
```

計算に使用できる総メモリ量は、1ノードあたりのメモリ量に、ジョブ実行にかかわる計算ノード数を乗じたものであるため、使用する計算ノード数を増やせばNの値(問題のサイズ)を増大させることができます。一般にNの値が大きいほど良い性能値が得られるため、Nの値は、使用計算ノード数ごとに以下のように設定しました。

* 1ノード: 20736
* 2ノード: 29184
* 3ノード: 36096
* 4ノード: 41088

P, Qの値は、可能な組み合わせのうち、最も良い性能が得られたものについて結果を示します。

### 1プロセス性能

1ノード1MPIプロセスでの測定結果は、以下の通りです。

|P|Q|MPIプロセス数|性能|
|-:|-:|-:|-:|
|1|1|1|84.3Gflops|

計算ノードのCPUは、`/proc/cpuinfo`によればIntel Xeon Platinum 8368とあるので、ここでは、この情報は物理マシン上のCPUをそのまま反映していると仮定します。

Xeon Platinum 8368はAVX-512をサポートしているため、64bit浮動小数点数を同時に8個保持できます($512 / 64$)。また、このCPUは1CPUあたりAVX-512のユニットを2ユニット持ちます。さらに、積和演算により、1命令で2回の演算が可能です。これにより、1サイクルあたりでの浮動小数点演算数は、

$$
  8 \times 2 \times 2 = 32
$$

です。

Xeon Platinum 8368のベース動作周波数は2.4GHzであるため、ベース動作周波数から計算した場合の1MPIプロセス(1CPU)での$R_{peak}$は、

$$
  2.4 \times 32 = 76.8 Gflops
$$

となります。このCPUはTurbo Boostをサポートしており、AVX-512使用時のTurbo Boost周波数は3.2GHzです(\*)。Turbo Boost周波数で計算した場合の$R_{peak}は、

$$
  3.2 \times 32 = 102.4 Gflops
$$

となります。

> (\*) このCPUのAVX-512使用時のTurbo Boost周波数は、一次情報としての仕様を参照するにはNDAが必要であるため、ここではインターネット上で第三者が書いている二次情報を参照しています。このため、正確でない可能性があります。

測定された性能の84.3Gflopsは、ベース動作周波数から計算した理論性能よりは高いため、Turbo Boostの効果は見られる、ということになります。

### ノード内並列化の効果

計算ノードには1ノードあたり3CPU存在するため、MPIプロセスを1ノードあたり3プロセス動作させることができます。計算ノードを1ノードだけ使用し、その中でMPIプロセスを複数動作させた場合の性能について示します。

|P|Q|MPIプロセス数|性能|1プロセス性能比|
|-:|-:|-:|-:|-:|
|1|2|2|158.8Gflops|1.88|
|1|3|3|221.9Gflops|2.63|

### ノード並列化の効果

使用したSlurmクラスタでは計算ノードを4ノード使用できるため、各ノードで1 MPIプロセスずつ動作させることで、ノード並列化の効果を確認します。なお、複数の計算ノードを使用するため、ノード数に対応して`HPL.dat`のNの値は変えています。

|計算ノード数|P|Q|MPIプロセス数|性能|1プロセス性能比|
|-:|-:|-:|-:|-:|-:|
|2|1|2|2|158.5Gflops|1.88|
|3|1|3|3|236.6Gflops|2.80|
|4|1|4|4|311.7Gflops|3.70|

### クラスタ全体を使用した場合の性能

今回使用したクラスタでは、最大で12MPIプロセスを動作させることができます。12MPIプロセス動作時の性能を示します。

|計算ノード数|P|Q|MPIプロセス数|性能|1プロセス性能比|
|-:|-:|-:|-:|-:|-:|
|4|1|12|12|740.7Gflops|8.79|

AVX-512使用時のTurbo Boost周波数から計算した場合のピーク性能と比較した場合の効率は、

$$
  740.7 \div (102.4 \times 12) = 0.6029
$$

となります。